In [ ]:
import os
import re
import pandas as pd
pd.options.display.max_colwidth = 10000

***WORKING WITH FILES***

In [ ]:
import glob #The glob module finds all the pathnames matching a specified pattern 
            # according to the rules used by the Unix shell

# r represents a raw string (instead of a unicoded string)
pathName = r'C:\Users\agupt\Documents\Courses\MGTF 423'


# fileName = 'C:\\Users\\agupt\\Documents\\Courses\\MGTF 423\\McKesson10K2022.txt'

for file in glob.glob(pathName+'/**/*.txt', recursive=True):
    print(file)

In [ ]:
fileName = 'McKesson10K2022.txt'

if not os.path.isfile(pathName + '\\' + fileName):
    print("{} does not exist ".format(pathName + '\\' + fileName))
reader = open(pathName +'\\'+ fileName, 'r')
outfile = open(pathName +'\\'+ 'clean_' + fileName,'w')
try:
    lines = reader.readlines() # gets the lines as a list
#    print(lines[0:20]) 
    for l in lines:
        t = l.replace('\n','').replace('?','')
        if len(t) > 0:
            # print(l)
            outfile.write(l)
finally:
    reader.close()
    outfile.close()

***STRING MATCHING -- SIMPLE VERSION***

In [ ]:
cleanFile = open(pathName +'\\'+ 'clean_' + fileName,'r')
content = cleanFile.read()
#list comprehension

matched_lines = [line.strip() for line in content.split('\n') if "Oncology" in line]
matched_lines


***USING DATA FRAMES TO ANALYZE TEXT DATA***

In [ ]:
def get_line_matching_substring(text, pattern):
    lines = text.splitlines()
    i = 0
    d = []
    for l in lines:
        if pattern in l:
            d.append([i,l.strip()])
            #print(d)
        i = i+1
    f = pd.DataFrame(d, columns = ['Line', 'Content'])
    return f

In [ ]:
match = get_line_matching_substring(content, "Oncology")
match

In [ ]:
somelines = match.loc[(match['Line'] >= 1000) & (match['Line'] <= 5000), ['Content']] # equivalent to 
                                                 # SELECT Content FROM match
                                                 # WHERE Line >= 1000 AND Line <= 5000
print(somelines)
#print(somelines.to_string(index = False, header = None))


***STRING MATCHING WITH REGULAR EXPRESSIONS***

Primary Reference: https://docs.python.org/3/library/re.html

In [ ]:
# Splitting a string

a="Beautiful, is; better*than\nugly"

re.split("; |, |\*|\n",a)

In [ ]:
# Extracting Sentences and words
myline = 'Noncontrolling Interests and Redeemable Noncontrolling Interests: Noncontrolling interests represent the portion of profit or loss, net assets, and comprehensive income or loss that is not allocable to McKesson Corporation. Net income attributable to noncontrolling interests includes recurring compensation that McKesson is obligated to pay to the noncontrolling shareholders of McKesson Europe AG (“McKesson Europe”), formerly known as Celesio AG, under the domination and profit and loss transfer agreement. Net income attributable to noncontrolling interests also includes third-party equity interests in the Company’s consolidated entities including Vantage Oncology Holdings, LLC (“Vantage”) and ClarusONE Sourcing Services LLP (“ClarusONE”), which was established between McKesson and Walmart, Inc in 2017. Noncontrolling interests with redemption features, such as put rights, that are not solely within the Company’s control are considered redeemable noncontrolling interests. Redeemable noncontrolling interests are presented outside of stockholders’ equity (deficit) in the Company’s Consolidated Balance Sheets. Refer to Financial Note 8, “Redeemable Noncontrolling Interests and Noncontrolling Interests,” for additional information.'

sentences = re.split(r'\. |:',myline)
sentences

words =  re.split(r'\. |:| |\.|“|”|\(|\)|,',myline)
words

In [ ]:
# Removing empty elements from a list - 1

clean_words = list(filter(None, words))
clean_words

In [ ]:
# Removing empty elements from a list - 2
# The join() method takes all items in an iterable like a list and joins them into one string 
# where the separator is specified. Here a space is the separator

clean_words2 = ' '.join(words).split()
clean_words2

In [ ]:
# Removing empty elements from a list - 2
# Physically iterate through the list and remove the null elements
while("" in words) :
    words.remove("")
  
print(words)

In [ ]:
# Removing duplicates from a list - 1
# Using *set

vector = [*set(words)]
print(vector)

In [ ]:
# Removing duplicates from a list - 2
# Using list comprehension

vector = []
[vector.append(x) for x in words if x not in vector]
print(vector)

In [ ]:
# Some standard regex functions

target_string = content
result = re.findall(r"[€£][0-9]* \w\w\w", target_string)
print(result)

target_string = content
result = re.search(r"[€£][0-9]* \w\w\w", target_string)
print(result.group())  

# group() returns the substring that was matched by the RE

a_sent = "Research and development expenses were  70 million, 74 million, and $96 million during 2022, 2021, and 2020, respectively."

# two groups enclosed in separate ( and ) bracket
result = re.search(r"([0-9]+) ([a-z]+)", a_sent)

# Extract matching values of all groups
print(result.groups())


# Extract match value of group 1
print(result.group(1))
print(result.group(2))

In [ ]:
# FINDING SECTION HEADERS

lines = content.split('\n')
for l in lines:
    s = re.search("^Item [0-9]+",l)
    if s != None:
        print(lines.index(l), l)
        
#for l in lines:
#    m = re.match("Item [0-9]+",l)
#    if m != None:
#        print(lines.index(l), l)

***DATAFRAMES AND REGULAR EXPRESSIONS***

In [ ]:
# A DataFrame for Text Data

lines = content.splitlines()
i = 0
j = 0
d = []
for l in lines:
    sent = re.split(r'\. |:',l)
    for s in sent:       
        d.append([i,j,s.strip()])
        j = j+1
    i = i+1
    j = 0
ldf = pd.DataFrame(d, columns = ['Line', 'Sent', 'Content'])
ldf

In [ ]:
regex = 'Item [0-9]*'
headers = ldf[ldf.Content.str.match(regex)]
headers

In [ ]:
regex = '\$500\s+[mb]illion'
money = ldf[ldf.Content.str.contains(regex)]
money

In [ ]:
regex = '[mb]illion'
money_ref = ldf[ldf.Content.str.contains(regex)]
money_ref

In [ ]:
ds1 = set(map(tuple, money_ref.values))
ds2 = set(map(tuple, money.values))

diff_frame = pd.DataFrame(list(ds1.difference(ds2)),columns = ['Line', 'Sent', 'Content'])
diff_frame

***INTRODUCTION TO TOKENIZATION***

In [ ]:
import nltk

In [ ]:
from nltk.tokenize import word_tokenize
text = "In April 2022, we entered into an amendment to the agreement for a purchase price of £110 million (or, approximately $144 million), including certain adjustments"
print(word_tokenize(text))

In [ ]:
from nltk.tokenize import WhitespaceTokenizer
Tokenizer=WhitespaceTokenizer()
print(Tokenizer.tokenize(text))

In [ ]:
from nltk.tokenize import LineTokenizer
LineTokenizer(blanklines='discard').tokenize(content) 

In [ ]:
from nltk.tokenize import wordpunct_tokenize
print(wordpunct_tokenize("I don't want to go"))
print(word_tokenize("I don't want to go"))

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

[word_tokenize(t) for t in sent_tokenize(content)] 

In [ ]:
# Tokenizing with Multi-word expressions (e.g., phrases)

from nltk.tokenize import MWETokenizer

tokenizer = MWETokenizer([('New', 'York'), ('the', 'White', 'House')])
sentence = 'Biden took a trip from the White House to New York'

tokenizer.tokenize(sentence.split())